STAT 453: Deep Learning (Spring 2020)  
Instructor: Sebastian Raschka (sraschka@wisc.edu)  

Course website: http://pages.stat.wisc.edu/~sraschka/teaching/stat453-ss2020/  
GitHub repository: https://github.com/rasbt/stat453-deep-learning-ss20

In [ ]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p torch

# Understanding Onehot Encoding and Cross Entropy in PyTorch

In [ ]:
import torch

## Onehot Encoding

In [ ]:
def to_onehot(y, num_classes):
    y_onehot = torch.zeros(y.size(0), num_classes)
    y_onehot.scatter_(1, y.view(-1, 1).long(), 1).float()
    return y_onehot

y = torch.tensor([0, 1, 2, 2])

y_enc = to_onehot(y, 3)

print('one-hot encoding:\n', y_enc)

## Softmax

Suppose we have some net inputs Z, where each row is one training example:

In [ ]:
Z = torch.tensor( [[-0.3,  -0.5, -0.5],
                   [-0.4,  -0.1, -0.5],
                   [-0.3,  -0.94, -0.5],
                   [-0.99, -0.88, -0.5]])

Z

Next, we convert them to "probabilities" via softmax:

$$P(y=j \mid z^{(i)}) = \sigma_{\text{softmax}}(z^{(i)}) = \frac{e^{z^{(i)}}}{\sum_{j=0}^{k} e^{z_{k}^{(i)}}}.$$

In [ ]:
def softmax(z):
    return (torch.exp(z.t()) / torch.sum(torch.exp(z), dim=1)).t()

smax = softmax(Z)
print('softmax:\n', smax)

The probabilties can then be converted back to class labels based on the largest probability in each row:

In [ ]:
def to_classlabel(z):
    return torch.argmax(z, dim=1)

print('predicted class labels: ', to_classlabel(smax))
print('true class labels: ', to_classlabel(y_enc))

## Cross Entropy

Next, we compute the cross entropy for each training example:

$$\mathcal{L}(\mathbf{W}; \mathbf{b}) = \frac{1}{n} \sum_{i=1}^{n} H(T_i, O_i),$$

$$H(T_i, O_i) = -\sum_m T_i \cdot log(O_i).$$

In [ ]:
def cross_entropy(softmax, y_target):
    return - torch.sum(torch.log(softmax) * (y_target), dim=1)

xent = cross_entropy(smax, y_enc)
print('Cross Entropy:', xent)

## In PyTorch

In [ ]:
import torch.nn.functional as F

Note that `nll_loss` takes log(softmax) as input:

In [ ]:
F.nll_loss(torch.log(smax), y, reduction='none')

Note that `cross_entropy` takes logits as input:

In [ ]:
F.cross_entropy(Z, y, reduction='none')

### Defaults

By default, nll_loss & cross_entropy are already returning the average over training examples, which is useful for stability during optimization.

In [ ]:
F.cross_entropy(Z, y)

In [ ]:
torch.mean(cross_entropy(smax, y_enc))